In [9]:
import os
import cv2
import numpy as np

In [2]:
path='./frames/im50.png'
img=cv2.imread(path)


In [3]:
import cv2
import numpy as np
# frame='002.png'
path='./frames/im50.png'
# load image as grayscale
img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

# img = cv2.applyColorMap(img, cv2.COLORMAP_JET)
# convert to 3 equal channels (only if img is already 1 channel grayscale)
img = cv2.merge((img, img, img))
# create 1 pixel red image
red = np.zeros((1, 1, 3), np.uint8)
red[:] = (0,0,255)

# create 1 pixel blue image
blue = np.zeros((2, 1, 3), np.uint8)
blue[:] = (255,0,0)

# append the two images
lut = np.concatenate((red, blue), axis=0)

# resize lut to 256 values
lut = cv2.resize(lut, (1,256), interpolation=cv2.INTER_CUBIC)

# apply lut
result = cv2.LUT(img, lut)


result=cv2.resize(result,(0,0),None, 0.4,0.4)
# save result
cv2.imwrite('001.png', result)


# display result
cv2.imshow('RESULT', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
def reorder(myPoints):
    pts=myPoints.shape
    myPointsNew=np.zeros_like(myPoints)
    myPoints=myPoints.reshape((pts[0],pts[2]))
    add=myPoints.sum(1)
    myPointsNew[0]=myPoints[np.argmin(add)]
    myPointsNew[3]=myPoints[np.argmax(add)]
    
    diff=np.diff(myPoints,axis=1)
    
    myPointsNew[1]=myPoints[np.argmin(diff)]
    myPointsNew[2]=myPoints[np.argmax(diff)]
    
    return myPointsNew


In [ ]:
import numpy as np
import cv2
from skimage import exposure 
path='./frames/6.jpeg'

# arrange the corridinates
def reorder(myPoints):
    pts=myPoints.shape
    myPointsNew=np.zeros_like(myPoints)
    myPoints=myPoints.reshape((pts[0],pts[2]))
    add=myPoints.sum(1)
    myPointsNew[0]=myPoints[np.argmin(add)]
    myPointsNew[3]=myPoints[np.argmax(add)]
    
    diff=np.diff(myPoints,axis=1)
    
    myPointsNew[1]=myPoints[np.argmin(diff)]
    myPointsNew[2]=myPoints[np.argmax(diff)]
    
    return myPointsNew

# Define the function to get color map
def getMap(frame,cThr=[100,100],minArea=20000,filter=0,draw=True):

    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    a_component = lab[:,:,1]
    th = cv2.threshold(a_component,140,255,cv2.THRESH_BINARY)[1]
    blur = cv2.GaussianBlur(th,(23,23), 29)
    heatmap_img = cv2.applyColorMap(blur, cv2.COLORMAP_JET)
    
    imgCanny= cv2.Canny(blur,cThr[0],cThr[1])
    kernal=np.ones((5,5))
    imgDial= cv2.dilate(imgCanny,kernal,iterations=3)
    imgThr=cv2.erode(imgDial,kernal,iterations=2)
    
    contours, hierarchy = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    finalCountours=[]
    for i in contours:
        area =cv2.contourArea(i)
        if area >minArea:
            peri=cv2.arcLength(i,True)
            approx=cv2.approxPolyDP(i,0.02*peri,True)
            bbox=cv2.boundingRect(approx)
            
            if filter >0:
                if len(approx)==filter:
                    finalCountours.append([len(approx),area,approx,bbox,i])
            else:
                finalCountours.append([len(approx),area,approx,bbox,i])

    finalCountours= sorted(finalCountours,key=lambda x:x[1],reverse=True)
    
    super_imposed_img = cv2.addWeighted(heatmap_img, 0.5, frame, 0.5, 0)
    return super_imposed_img,finalCountours

capture = cv2.VideoCapture(path)
while True:
    ret, frame = capture.read()
    scale =3
    wroad=210 * scale
    hroad=297*scale
    
    if ret:
        # Resize original frame
        frame = cv2.resize(frame, (580, 550)) 
        cam,conts = getMap(frame,minArea=90000,draw=False)
        
        map_img = exposure.rescale_intensity(cam, out_range=(0, 255))
        map_img = np.uint8(map_img)
        heatmap_img = cv2.applyColorMap(map_img, cv2.COLORMAP_JET)

        # Merge map and frame
        fin = cv2.addWeighted(heatmap_img, 0.9, frame, 0.9, 0.1)
        print(len(conts))
        if len(conts)!=0:
            
            biggest=conts[0][2]
            print(biggest)
            cv2.drawContours(frame, [biggest], -1, (0, 0, 255), 2)
            cv2.imshow('road',frame)

        # save result
        cv2.imwrite('00q2.png', fin)
        cv2.imshow('frame', fin)
        cv2.waitKey(0)

        if cv2.waitKey(0) & 0xFF == ord('q'):
            break

capture.release()
cv2.destroyAllWindows()


1
[[[247 122]]

 [[137 308]]

 [[114 486]]

 [[450 546]]

 [[530 428]]

 [[474 231]]]
